In [27]:
from lxml import html  
import csv,os,json
import requests
from time import sleep
import ast
import re
from selenium import webdriver
import pandas as pd

In [51]:
driver = webdriver.Chrome(executable_path=r'C:\Geckodriver\chromedriver.exe')
driver.set_page_load_timeout(500)

In [100]:
retries = 0
def WalmartParser(url):
    global retries
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    driver.get(url)
    #print(driver)
    page = driver.page_source
    rpage = requests.get(url,headers=headers)
   
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page)
            script_data = doc.xpath('//script[contains(., "Specifications")]/text()')[0]
            
            XPATH_NAME = '//h1[contains(@class,"prod-ProductTitle")]/div/text()'
            XPATH_SALE_PRICE = '//span[@class="display-inline-block-xs prod-PaddingRight--s"]//text()'
            XPATH_CATEGORY = '//ol[@class="breadcrumb-list "]//text()'
            XPATH_RATING = '//span[@class="seo-avg-rating" and @itemprop="ratingValue"]/text()'
             
            
            ''' 
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
            '''
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            RAW_RATING = doc.xpath(XPATH_RATING)
            print(RAW_SALE_PRICE)
           # RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            '''
            
            
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            '''
            
            AUTHOR, PUBLISHER, LANGUAGE, PAGES, ISBN10, DIMENSIONS = None, None, None, None, None, None
            

            data = re.search(r'"Specifications":(.*?),"ShortDescription', script_data).group(1)
            #print(data)
            #import json
            #data = json.loads(data)
            data = ast.literal_eval(data)
            data = (data["specifications"]["values"][0])
            for item in data:
                #print item
                for k,v in item.items():
                    if k == "author":
                        RAW_AUTHOR = v['values']
                        AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
                        
                    elif k == "publisher":
                        RAW_PUBLISHER = v['values']
                        PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None

                    elif k == "languages":
                        RAW_LANGUAGE = v['values']
                        LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
                        
                    elif k == "number_of_pages":
                        RAW_PAGES = v['values']
                        PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None

                    elif k == "isbn-10":
                        RAW_ISBN10 = v['values']
                        ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
                    elif k == "dimension":
                        RAW_DIMENSIONS = v['values']
                        DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
                    
            #AUTHOR = data
            
            #print AUTHOR
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            
           
            
            #print AUTHOR, PAGES, PUBLISHER
            '''
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
                '''
 
            if rpage.status_code!=200 or not NAME or NAME == "":
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                   # 'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    #'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
            retries = 0
            return data
        except Exception as e:
            retries = retries + 1
            if retries > 5:
                retries = 0
                return ""
            print("Retrying with exception , retires = " + str(retries))
            driver.get(url)
            #print(driver)
            rpage = requests.get(url,headers=headers)
            page = driver.page_source
 

In [101]:
extracted_data = []
def ReadAsin(AsinList):
    count = 0
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    for i in AsinList:
        url = "http://www.walmart.com/ip/"+i
        print("Processing: "+url)
        json_data = WalmartParser(url)
        if json_data != "":
            extracted_data.append(json_data)
        else:
            print("Skipping")
        count += 1
        #sleep(5)
        print(count)
    f=open('data_walmart.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [102]:
formatted_asin = []
df = pd.read_csv("walmart_product_asins.csv")
for index, row in df.iterrows():
    #print(row[1])
    formatted_asin.append(str(row[1]))

In [103]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[200:1000])

Processing: http://www.walmart.com/ip/834005731
Retrying with exception , retires = 1
Retrying with exception , retires = 2
Retrying with exception , retires = 3
Retrying with exception , retires = 4
Retrying with exception , retires = 5
Skipping
1
Processing: http://www.walmart.com/ip/5664513
['$', '15', '.', '11']
2
Processing: http://www.walmart.com/ip/53398236
['$', '14', '.', '69']
3
Processing: http://www.walmart.com/ip/44372468
['$', '7', '.', '19']
4
Processing: http://www.walmart.com/ip/655170252
['$', '17', '.', '10']
5
Processing: http://www.walmart.com/ip/55564639
['$', '7', '.', '75']
6
Processing: http://www.walmart.com/ip/56176553
['$', '5', '.', '98']
7
Processing: http://www.walmart.com/ip/27208141
['$', '13', '.', '06']
8
Processing: http://www.walmart.com/ip/51799379
['$', '4', '.', '97']
9
Processing: http://www.walmart.com/ip/15998422
['$', '8', '.', '66']
10
Processing: http://www.walmart.com/ip/56141697
['$', '10', '.', '87']
11
Processing: http://www.walmart.com

['$', '10', '.', '66']
111
Processing: http://www.walmart.com/ip/56155946
['$', '12', '.', '92']
112
Processing: http://www.walmart.com/ip/592238819
['$', '8', '.', '88']
113
Processing: http://www.walmart.com/ip/37367003
['$', '10', '.', '03']
114
Processing: http://www.walmart.com/ip/11593343
['$', '22', '.', '40']
115
Processing: http://www.walmart.com/ip/55599900
['$', '6', '.', '22']
116
Processing: http://www.walmart.com/ip/873191033
['$', '9', '.', '99']
117
Processing: http://www.walmart.com/ip/52289579
['$', '4', '.', '99']
118
Processing: http://www.walmart.com/ip/471784
['$', '16', '.', '82']
119
Processing: http://www.walmart.com/ip/56101436
['$', '5', '.', '01']
120
Processing: http://www.walmart.com/ip/39391217
['$', '4', '.', '99']
121
Processing: http://www.walmart.com/ip/44977390
['$', '9', '.', '91']
122
Processing: http://www.walmart.com/ip/53821688
['$', '10', '.', '35']
123
Processing: http://www.walmart.com/ip/41265656
['$', '9', '.', '57']
124
Processing: http://

['$', '6', '.', '99']
221
Processing: http://www.walmart.com/ip/11043046
['$', '7', '.', '68']
222
Processing: http://www.walmart.com/ip/56101479
['$', '10', '.', '18']
223
Processing: http://www.walmart.com/ip/21085496
['$', '7', '.', '98']
224
Processing: http://www.walmart.com/ip/56139528
['$', '12', '.', '61']
225
Processing: http://www.walmart.com/ip/20560407
['$', '9', '.', '93']
226
Processing: http://www.walmart.com/ip/55959152
['$', '7', '.', '48']
227
Processing: http://www.walmart.com/ip/56175307
['$', '9', '.', '40']
228
Processing: http://www.walmart.com/ip/14017295
['$', '22', '.', '14']
229
Processing: http://www.walmart.com/ip/17800359
['$', '9', '.', '99']
230
Processing: http://www.walmart.com/ip/46326711
['$', '8', '.', '48']
231
Processing: http://www.walmart.com/ip/36957240
['$', '5', '.', '95']
232
Processing: http://www.walmart.com/ip/48726344
['$', '8', '.', '78']
233
Processing: http://www.walmart.com/ip/46127592
['$', '5', '.', '52']
234
Processing: http://www

['$', '11', '.', '71']
332
Processing: http://www.walmart.com/ip/51795617
['$', '9', '.', '59']
333
Processing: http://www.walmart.com/ip/33936420
['$', '16', '.', '65']
334
Processing: http://www.walmart.com/ip/13277252
['$', '10', '.', '18']
335
Processing: http://www.walmart.com/ip/56144397
['$', '10', '.', '38']
336
Processing: http://www.walmart.com/ip/43906139
['$', '10', '.', '26']
337
Processing: http://www.walmart.com/ip/52285219
['$', '11', '.', '57']
338
Processing: http://www.walmart.com/ip/40566061
['$', '9', '.', '09']
339
Processing: http://www.walmart.com/ip/48449514
['$', '7', '.', '03']
340
Processing: http://www.walmart.com/ip/51489010
['$', '11', '.', '33']
341
Processing: http://www.walmart.com/ip/46226068
['$', '8', '.', '38']
342
Processing: http://www.walmart.com/ip/6021139
['$', '3', '.', '12']
343
Processing: http://www.walmart.com/ip/43911798
['$', '19', '.', '55']
344
Processing: http://www.walmart.com/ip/56143011
['$', '9', '.', '40']
345
Processing: http:/

['$', '12', '.', '94']
443
Processing: http://www.walmart.com/ip/55220279
['$', '7', '.', '99']
444
Processing: http://www.walmart.com/ip/55697971
['$', '5', '.', '64']
445
Processing: http://www.walmart.com/ip/55740861
['$', '7', '.', '48']
446
Processing: http://www.walmart.com/ip/54132173
['$', '8', '.', '98']
447
Processing: http://www.walmart.com/ip/12309416
['$', '10', '.', '58']
448
Processing: http://www.walmart.com/ip/55818568
['$', '5', '.', '78']
449
Processing: http://www.walmart.com/ip/55970984
['$', '8', '.', '33']
450
Processing: http://www.walmart.com/ip/55676279
['$', '7', '.', '41']
451
Processing: http://www.walmart.com/ip/56120883
['$', '4', '.', '99']
452
Processing: http://www.walmart.com/ip/55218808
['$', '9', '.', '40']
453
Processing: http://www.walmart.com/ip/4767023
['$', '9', '.', '00']
454
Processing: http://www.walmart.com/ip/19860312
['$', '8', '.', '52']
455
Processing: http://www.walmart.com/ip/36957317
['$', '58', '.', '88']
456
Processing: http://www.

['$', '10', '.', '91']
555
Processing: http://www.walmart.com/ip/13920784
['$', '3', '.', '99']
556
Processing: http://www.walmart.com/ip/46127953
['$', '9', '.', '97']
557
Processing: http://www.walmart.com/ip/29142928
['$', '5', '.', '49']
558
Processing: http://www.walmart.com/ip/471785
['$', '16', '.', '06']
559
Processing: http://www.walmart.com/ip/54978623
['$', '9', '.', '77']
560
Processing: http://www.walmart.com/ip/55220277
['$', '14', '.', '06']
561
Processing: http://www.walmart.com/ip/55059364
['$', '11', '.', '33']
562
Processing: http://www.walmart.com/ip/48584490
['$', '7', '.', '48']
563
Processing: http://www.walmart.com/ip/5511356
['$', '8', '.', '04']
564
Processing: http://www.walmart.com/ip/55959153
['$', '6', '.', '99']
565
Processing: http://www.walmart.com/ip/46848400
['$', '7', '.', '99']
566
Processing: http://www.walmart.com/ip/56172778
['$', '19', '.', '69']
567
Processing: http://www.walmart.com/ip/23733767
['$', '5', '.', '90']
568
Processing: http://www.

['$', '8', '.', '86']
666
Processing: http://www.walmart.com/ip/47696725
['$', '12', '.', '18']
667
Processing: http://www.walmart.com/ip/552840699
['$', '15', '.', '40']
668
Processing: http://www.walmart.com/ip/20610503
['$', '36', '.', '82']
669
Processing: http://www.walmart.com/ip/29268081
['$', '16', '.', '59']
670
Processing: http://www.walmart.com/ip/12429615
['$', '7', '.', '88']
671
Processing: http://www.walmart.com/ip/56044883
['$', '9', '.', '74']
672
Processing: http://www.walmart.com/ip/56088858
['$', '9', '.', '40']
673
Processing: http://www.walmart.com/ip/20610504
['$', '35', '.', '96']
674
Processing: http://www.walmart.com/ip/52705553
['$', '9', '.', '48']
675
Processing: http://www.walmart.com/ip/43378098
['$', '7', '.', '61']
676
Processing: http://www.walmart.com/ip/56118005
['$', '9', '.', '63']
677
Processing: http://www.walmart.com/ip/55421916
['$', '8', '.', '12']
678
Processing: http://www.walmart.com/ip/55804116
['$', '14', '.', '11']
679
Processing: http:/

In [24]:
print(len(extracted_data))

201


In [29]:
f=open('data_walmart.json','w')
json.dump(extracted_data,f,indent=4)
f.close()